<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
    <i>Aprender una técnica no es un fin en si mismo, simplemente indica por donde hay que empezar</i><br><b>Proverbio Japonés</b></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>R</strong>etrieval <strong>A</strong>ugmented <strong>G</strong>eneration</p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Cargar librerías</strong></p> 

In [1]:
import warnings
import os
import numpy as np
import json
import torch # Necesario para manejar dtype y device_map
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
warnings.filterwarnings("ignore")
#Ajustar path
os.environ["HF_HOME"] = "D:/HF/MODEL_CACHE"   
os.environ["HF_DATASETS_CACHE"] = "D:/HF/MODEL_CACHE"

In [2]:
!pip install faiss-cpu

In [3]:
import faiss

<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>faiss:</strong> búsqueda por similitud en vectores - Doc: 
<a href="https://github.com/facebookresearch/faiss?tab=readme-ov-file" target=\"_blank\">Faiss</a></p>

In [4]:
!pip install sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer

<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>sentence_transformers:</strong> generador de embedding - Doc: 
<a href="https://www.sbert.net/" target=\"_blank\">SentenceTransformers Documentation</a></p>

In [6]:
!pip install transformers

In [7]:
from transformers import pipeline

<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>pipeline:</strong> secuencias - Doc:    
<a href="https://huggingface.co/docs/transformers/main_classes/pipelines" target=\"_blank\">Pipelines Method</a><br>
<a href="https://huggingface.co/docs/transformers/v4.46.0/en/add_new_pipeline" target=\"_blank\">How to create a custom pipeline?</a></p>

In [8]:
!pip install accelerate

In [9]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Definición del Corpus de Documentos - Base de conocimiento</strong></p> 

In [10]:
# Carga de documentos desde JSONL
print("Cargando documentos desde JSONL...")
documents = []
with open(r"C:\Users\agarmendia\Desktop\PNCIL_TinyLlama\pncil_documents.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
        doc = json.loads(line)
        documents.append(doc['text'])

print(f"✓ Cargados {len(documents)} documentos")
print(f"\nPrimeros 200 caracteres del primer documento:")
print(documents[0][:200])

Cargando documentos desde JSONL...
✓ Cargados 58 documentos

Primeros 200 caracteres del primer documento:
CMO es Certificado Médico Oficial. Se tramita a través de la Agencia Nacional de Discapacidad (ANDIS) y es un requisito obligatorio para solicitar las Pensiones No Contributivas por Invalidez Laboral 


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>1. Indexación: Creación de la Base de Conocimiento Vectorial</strong></p> 
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>1.1. Cargar el Modelo de Embeddings - Entrenamiento Heredado</strong><br> 
<strong>all-MiniLM-L6-v2:</strong> Mapea oraciones y párrafos a un espacio vectorial de 384 dimensiones - 
Doc: <a href="https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2" target=\"_blank\">all-MiniLM-L6-v2</a></p>

In [11]:
print("Cargando modelo de SentenceTransformer - all-MiniLM-L6-v2 ...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
dimension = 384  # Se igualan las diemnsiones vectoriales al modelo utilizado
print("Modelo cargado")

Cargando modelo de SentenceTransformer - all-MiniLM-L6-v2 ...
Modelo cargado


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>1.2. Generar Embeddings para los Documentos</strong><br></p>

In [12]:
print("Generando embeddings para el corpus...")
doc_embeddings = embedding_model.encode(documents)
doc_embeddings = np.array(doc_embeddings, dtype='float32')
print(f"Embeddings generados. Forma de la matriz: {doc_embeddings.shape}")

Generando embeddings para el corpus...
Embeddings generados. Forma de la matriz: (58, 384)


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>1.3. Crear y Poblar el Índice FAISS</strong><br></p>

In [13]:
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)
print(f"Índice FAISS creado y poblado. Contiene {index.ntotal} vectores.")

Índice FAISS creado y poblado. Contiene 58 vectores.


<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>2. Generación: Carga del Modelo de Lenguaje</strong></p> 

In [14]:
# Cargar TinyLlama-1.1B-Chat-v1.0

print("Cargando modelo generativo (TinyLlama-1.1B-Chat)...")
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" 

# 1. Cargar Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 2. Cargar Modelo (Se usa float16/bfloat16 si hay GPU o si la CPU lo soporta bien, si no, se deja la default)
# Como la inferencia en CPU es más lenta, se podría usar device_map="cpu" explícitamente o dejarlo en -1
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto" # Intenta usar GPU, si no, usa CPU. Para forzar CPU usar device_map="cpu"
)

# 3. Crear Pipeline
generator = pipeline(
    task="text-generation", 
    model=model, 
    tokenizer=tokenizer,
    #device=-1 # Forzar el uso de CPU para la demo si la GPU no es potente o no existe.
)
print("Modelo generativo TinyLlama cargado.")

Cargando modelo generativo (TinyLlama-1.1B-Chat)...


Device set to use cpu


Modelo generativo TinyLlama cargado.


<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>3. Método de extracción de vectores similares</strong></p> 

In [15]:
def retrieve_documents(query: str, k: int = 6, min_similarity: float = 0.4):
    """
    Recupera documentos relevantes con filtrado por similitud mínima.
    
    Args:
        query: consulta del usuario
        k: número máximo de documentos a recuperar
        min_similarity: umbral mínimo de similitud coseno (0-1)
    """
    # Embedding de la query
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)  
    
    # Buscar en FAISS
    distances, indices = index.search(query_embedding, k)

    similarities = 1 / (1 + distances[0])  # conversión aproximada
    
    # FILTRAR por threshold
    filtered_docs = []
    for i, sim in enumerate(similarities):
        if sim >= min_similarity:
            filtered_docs.append(documents[indices[0][i]])
    
    return filtered_docs   

<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>4. Método de generación de texto</strong><br> 
<p style="font-size:14px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong><u>Generator Params</u></strong>:
<ul style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<li><i><b>document:</b></i> texto base</li>
<li><i><b>max_new_tokens:</b></i> genera hasta NN tokens</li>
<li><i><b>num_return_sequences:</b></i> retorna solo un texto</li>
<li><i><b>temperature:</b></i> ajusta la creatividad del modelo</li>
<li><i><b>top_k y top_p:</b></i> controlan la selección de palabras para una generación más coherente y variada</li>
</ul></p>

In [16]:
# Ajustar Prompt y Parámetros
def generate_answer(query: str, retrieved_docs: list):
    
    context = " ".join(retrieved_docs)

    # El context window de TinyLlama es 2048. Se limita a 2000.
    context_limit = 2000 
    if len(context) > context_limit:
        context = context[:context_limit] + "..."
    
    # Construir Prompt en formato Llama Chat Template
    system_instruction = "Eres un asistente de preguntas y respuestas cuya ÚNICA fuente de conocimiento es el 'Contexto' proporcionado. Debes responder la 'Pregunta' basandote estrictamente en el contexto, extrae SOLAMENTE esa información. Si el contexto NO PERMITE una respuesta fidedigna, debes responder EXCLUSIVAMENTE: 'No tengo información suficiente para responder a esa pregunta.'."
   
    user_prompt = f"Contexto: {context}\n\nPregunta: {query}"
    
    # Formato Llama Chat: <s>[INST] <<SYS>> {system_instruction} <</SYS>> {user_prompt} [/INST]
    prompt = f"<s>[INST] <<SYS>> {system_instruction} <</SYS>> {user_prompt} [/INST]"

    # Generar la respuesta (Parámetros ajustados para la facticidad)
    generated_output = generator(
        prompt,
        max_new_tokens=256,
        num_return_sequences=1,
        truncation=True,
        # Temperatura BAJA para reducir la alucinación 
        temperature=0.4,
        top_p=0.90,
        top_k=40
    )
    
    answer = generated_output[0]['generated_text']
    
    # Limpieza: Llama devuelve el prompt completo, hay que limpiarlo
    if "[/INST]" in answer:
        # Extraemos solo el texto después del tag de cierre de instrucción
        cleaned_answer = answer.split("[/INST]")[1].strip()

        # Eliminar el tag de sistema si el modelo lo regeneró por error
        if "<<SYS>>" in cleaned_answer:
            cleaned_answer = cleaned_answer.split("<<SYS>>")[1].strip()
            
        # Eliminar en la palabra 'Pregunta' 
        if "Pregunta:" in cleaned_answer:
            cleaned_answer = cleaned_answer.split("Pregunta:")[0].strip()

        # Añadir o verificar esta línea en la sección de limpieza 
        if cleaned_answer.lower().startswith("respuesta:"):
            cleaned_answer = cleaned_answer[len("respuesta:"):].strip()

        # Cortar en el tag de fin de secuencia '</s>'
        if "</s>" in cleaned_answer:
            cleaned_answer = cleaned_answer.split("</s>")[0].strip()
            
    else:
        # Fallback si el modelo no genera el tag de cierre 
        cleaned_answer = answer.strip()
    
    return cleaned_answer

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Aclaración de Uso:</strong><br> 
I. El conjunto de documentos es un conjunto reducido a modo de ejemplo<br>
II. Los parametros de generator afectan directamente a la respuesta del modelo    
</p>

<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>5. Ejemplo de Uso I:</strong><br> 
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Hacer pregunta</strong>:
</p>

In [17]:
query_1 = "¿Que es TAD?"

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Recuperar documentos relevantes</strong>:
</p>

In [18]:
relevant_docs_1 = retrieve_documents(query_1, k=2)
print("Documentos Recuperados:")
for i, doc in enumerate(relevant_docs_1):
    print(f"{i+1}. {doc}")

Documentos Recuperados:
1. ANDIS es Agencia Nacional de Discapacidad. TAD es Plataforma Trámites a Distancia.
2. ANDIS es Agencia Nacional de Discapacidad. TAD es Plataforma Trámites a Distancia.


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Generar respuesta en función de los documentos</strong>:
</p>

In [19]:
answer_1 = generate_answer(query_1, relevant_docs_1)
print("Respuesta Generada:")
print(f"{answer_1}")

Respuesta Generada:
TAD es Plataforma Trámites a Distancia.


<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>6. Ejemplo de Uso II:</strong><br> 
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Hacer pregunta</strong>:
</p>

In [20]:
query_2 = "¿Cuales son los pasos a seguir después de la auditoría?"


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Recuperar documentos relevantes</strong>:
</p>

In [21]:
relevant_docs_2 = retrieve_documents(query_2, k=2)
print("Documentos Recuperados:")
for i, doc in enumerate(relevant_docs_2):
    print(f"{i+1}. {doc}")

Documentos Recuperados:
1. MENORES DE EDAD Y AUDITORÍA Las personas menores de edad NO están incluidas en esta auditoría de pensiones.
2. Pasos a seguir después de la auditoría: enviar a ANDIS la documentación y el certificado recibido por el prestador de PAMI dentro de los 30 días posteriores a la cita, escaneados (no original) por alguno de los siguientes medios: Plataforma de Trámites a Distancia, presencialmente en las oficinas de ANSES con turno previo, en los Centros de atención de la Agencia Nacional de Discapacidad, por correo postal a Hipólito Yrigoyen N°1447 CP 1089 CABA.


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Generar respuesta en función de los documentos</strong>:
</p>

In [22]:
answer_2 = generate_answer(query_2, relevant_docs_2)
print("Respuesta Generada:")
print(f"{answer_2}")

Respuesta Generada:
Enviar a ANDIS la documentación y el certificado recibido por el prestador de PAMI dentro de los 30 días posteriores a la cita, escaneados (no original) por alguno de los siguientes medios: Plataforma de Trámites a Distancia, presencialmente en las oficinas de ANSES con turno previo, en los Centros de atención de la Agencia Nacional de Discapacidad, por correo postal a Hipólito Yrigoyen N°1447 CP 1089 CABA.


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Reflexión:</strong><br> 
El corpus de documentos provisto es el que aprota el contexto sobre el cual va a ser constuida la respuesta 
</p>

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Actividad:</strong><br> 
Modificar documentos y peguntas, junto a parametros y evaluar resultado obtenido   
</p>

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 